### Loading packages

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import cv2 as cv
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn.metrics as metrics
import scipy.cluster.hierarchy as Sciplot
import shap
import os
import glob
import joblib
from statsmodels.stats.outliers_influence import variance_inflation_factor

### Loading FPA FOD

In [ ]:
FPA_FOD_P = pd.read_csv(filepath_or_buffer = '/bsuhome/yavarpourmohamad/scratch/Summer_2022/Aggregated/FPA_FOD_Plus.csv',
                        sep = ',', low_memory = False)

### Selecting State and features

In [ ]:
states = ['WA', 'OR', 'CA', 'ID', 'NV', 'MT', 'WY', 'UT', 'AZ', 'CO', 'NM']
FPA_FOD_W = FPA_FOD_P.loc[FPA_FOD_P['STATE'].isin(states)]
FPA_FOD_W = FPA_FOD_W.loc[FPA_FOD_W['LONGITUDE'] < -102.05]

cols = ['DISCOVERY_DOY', 'FIRE_YEAR', 'STATE', 'FIPS_CODE', 'NWCG_GENERAL_CAUSE', 'Annual_etr', 'Annual_precipitation',
        'Annual_tempreture', 'pr', 'tmmn', 'vs', 'fm100', 'fm1000', 'bi', 'vpd', 'erc', 'Elevation_1km', 'Aspect_1km', 'erc_Percentile', 'Slope_1km',
        'TPI_1km', 'EVC', 'Evacuation', 'SDI', 'FRG', 'No_FireStation_5.0km', 'Mang_Name', 'GAP_Sts', 'GACC_PL', 'GDP', 'GHM', 'NDVI-1day', 'NPL',
        'Popo_1km', 'road_county_dis', 'road_interstate_dis', 'road_common_name_dis', 'road_other_dis', 'road_state_dis', 'road_US_dis', 'RPL_THEMES',
        'RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4']
FPA_FOD_c = FPA_FOD_W[cols]

print('Before cleaning:')
print('Rows: ', FPA_FOD_c.shape[0])
print('columns: ', FPA_FOD_c.shape[1])

### Data cleaning

In [ ]:
# Road
road_cols = ['road_county_dis', 'road_interstate_dis', 'road_common_name_dis', 'road_other_dis', 'road_state_dis', 'road_US_dis']
FPA_FOD_c['Distance2road'] = FPA_FOD_c[road_cols].min(axis = 1)
FPA_FOD_r = FPA_FOD_c.drop(labels = road_cols, axis = 1)
FPA_FOD_r = FPA_FOD_r.dropna(axis = 0, subset = ['Distance2road'])

FPA_FOD_r.reset_index(drop = True, inplace = True)

# Precentiles
pers = ['erc_Percentile']
for per in pers:
     temp = FPA_FOD_r['erc_Percentile'].str.split(pat = '-', expand = True)
     if temp.shape[1] == 2:
          lower = temp[0].str.replace(pat = '>', repl = '').str.replace(pat = '%', repl = '').str.replace(pat = '<', repl = '').astype(float)
          upper = temp[1].str.replace(pat = '>', repl = '').str.replace(pat = '%', repl = '').str.replace(pat = '<', repl = '').astype(float)
          FPA_FOD_r[per] = np.mean(pd.concat(objs = [lower, upper], axis = 1), axis = 1)
     else:
          FPA_FOD_r[per] = temp[0].str.replace(pat = '>', repl = '').str.replace(pat = '%', repl = '').astype(float)

# Management type
j = 0
labels = dict()
for i in FPA_FOD_r['Mang_Name'].unique():
     labels[i] = j
     j+=1

for i in FPA_FOD_r['Mang_Name'].unique():
        FPA_FOD_r.loc[FPA_FOD_r['Mang_Name'] == i, 'Mang_Name'] = int(labels[i])
print('\nManagement type code:\n', labels, '\n')

FPA_FOD_r.reset_index(drop = True, inplace = True)

FPA_FOD_r = FPA_FOD_r.fillna(value = 0)

# # GHM
# FPA_FOD_r[FPA_FOD_r['GHM'] < 0] = 0

print('After cleaning:')
print('Rows: ', FPA_FOD_r.shape[0])
print('columns: ', FPA_FOD_r.shape[1])

# FPA_FOD_r.to_csv(path_or_buf = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/FPA_FOD_west.csv',
#                  sep = ',', index = False)
# FPA_FOD_r = FPA_FOD_r.drop(labels = 'STATE', axis = 1)

### Preparing to feed ML

In [ ]:
FPA_FOD_r = pd.read_csv(filepath_or_buffer = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/FPA_FOD_west.csv',
                        sep = ',', low_memory = False)
FPA_FOD_r = FPA_FOD_r.drop(labels = 'STATE', axis = 1)
unseen_data = FPA_FOD_r[FPA_FOD_r['NWCG_GENERAL_CAUSE'] == 'Missing data/not specified/undetermined']
data = FPA_FOD_r[FPA_FOD_r['NWCG_GENERAL_CAUSE'] != 'Missing data/not specified/undetermined']

j = 0
labels = dict()
for i in data['NWCG_GENERAL_CAUSE'].unique():
     labels[i] = j
     j+=1

for i in data['NWCG_GENERAL_CAUSE'].unique():
        data.loc[data['NWCG_GENERAL_CAUSE'] == i, 'NWCG_GENERAL_CAUSE'] = int(labels[i])
print(labels)

data = data.astype(np.float64)

data_X = data.loc[:, data.columns != 'NWCG_GENERAL_CAUSE']
data_y = data.loc[:, data.columns == 'NWCG_GENERAL_CAUSE']

In [ ]:
corr = FPA_FOD_r.corr()
corr.to_csv(path_or_buf = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/Corr.csv',
            sep = ',',
            index = True)

### Variance Inflation Factor (VIF)

In [ ]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = data_X.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(data_X.values, i)
						for i in range(len(data_X.columns))]

vif_data.to_csv(path_or_buf = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/vif_west.csv', sep = ',')

### Train, Validation, and Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size = 0.2, random_state = 42)
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2)

y_train = pd.to_numeric(y_train['NWCG_GENERAL_CAUSE'])
# y_valid = pd.to_numeric(y_valid['NWCG_GENERAL_CAUSE'])
y_test = pd.to_numeric(y_test['NWCG_GENERAL_CAUSE'])

print('train_X size: ', len(X_train))
# print('valid_X size: ', len(X_valid))
print('test_X size: ', len(X_test))

### train, evaluation, and text distribution

In [ ]:
print(labels)

# fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
fig, (ax1, ax3) = plt.subplots(1, 2, figsize=(15, 5))

ax1 = sns.histplot(data = y_train,
                   stat = 'percent',
                   discrete = True,
                   kde = False,
                   ax = ax1)
ax1.grid()
ax1.set_title('Train portion')
ax1.bar_label(ax1.containers[0])

# ax2 = sns.histplot(data = y_valid,
#                    stat = 'percent',
#                    discrete = True,
#                    kde = False,
#                    ax = ax2)
# ax2.grid()
# ax2.set_title('Valid portion')
# ax2.bar_label(ax2.containers[0])

ax3 = sns.histplot(data = y_test,
                   stat = 'percent',
                   discrete = True,
                   kde = False,
                   ax = ax3)
ax3.grid()
ax3.set_title('Test portion')
ax3.bar_label(ax3.containers[0])

### Calculating class weights

In [ ]:
# cls_wght = dict()
# for cls in data.NWCG_GENERAL_CAUSE.unique():
#      cls_wght[cls] = len(data)/(len(data.NWCG_GENERAL_CAUSE.unique()) * len(data[data['NWCG_GENERAL_CAUSE'] == cls]))
# cls_wght = class_weight.compute_class_weight(class_weight = 'balanced',
#                                              classes = data.NWCG_GENERAL_CAUSE.unique(),
#                                              y = data.NWCG_GENERAL_CAUSE)
cls_wght = class_weight.compute_sample_weight(class_weight = 'balanced',
                                              y = y_train)

### With Class Weight

The hyper parameters for XGboost were optimized using GridsearchCV.

You can either train the model using this hyper parameters, or load the trained model.

In [ ]:
# clf_XGb_ww = xgb.XGBClassifier(tree_method = 'hist',
#                             base_score = 0.5,
#                             booster = 'gbtree',
#                             objective = 'multi:softprob',
#                             num_class = 12,
#                             max_depth = 14,
#                             alpha = 0,
#                             learning_rate = 0.08,
#                             n_estimators = 1550,
#                             seed = 42,
#                             )
# clf_XGb_ww.fit(X_train, y_train, sample_weight = cls_wght)

In [ ]:
# joblib.dump(value = clf_XGb_ww,
#             filename = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/clf_xgb_WW.pkl')
clf_XGb_ww = joblib.load(filename = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/clf_xgb_WW.pkl')

In [ ]:
print("Accuracy on test: %.2f%%" % (clf_XGb_ww.score(X = X_test, y = y_test) * 100.0))

In [ ]:
def colorcode_confusion_matrix(df_cm, title):
    cmap = plt.cm.Reds

    plt.figure(figsize = (8, 6))
    plt.imshow(df_cm, cmap = cmap)
    plt.colorbar()

    for i in range(len(df_cm.index)):
        for j in range(len(df_cm.columns)):
            plt.text(i, j, f"{df_cm.iloc[i, j]:.2f}", ha = "center", va = "center", fontsize = 9) # Print decimal values
            # plt.text(i, j, f"{df_cm.iloc[i, j]}", ha = "center", va = "center", fontsize = 9) # Print integer values

    plt.xticks(np.arange(len(df_cm.columns)), df_cm.columns, rotation = 90)
    plt.yticks(np.arange(len(df_cm.columns)), df_cm.index)

    plt.title(title)
    plt.show()

In [ ]:
pred_y_test = clf_XGb_ww.predict(X = X_test)
cm_test = confusion_matrix(y_true = y_test,
                           y_pred = pred_y_test,
                           labels = list(labels.values()),
                           normalize = 'true')
cm_test = pd.DataFrame(data = cm_test, index = list(labels.keys()), columns = list(labels.keys()))

colorcode_confusion_matrix(df_cm = cm_test, title = 'Confusion matrix on test set')

### Without Class Weight

The hyper parameters for XGboost were optimized using GridsearchCV.

You can either train the model using this hyper parameters, or load the trained model.

In [ ]:
# clf_XGb = xgb.XGBClassifier(tree_method = 'hist',
#                             base_score = 0.5,
#                             booster = 'gbtree',
#                             objective = 'multi:softprob',
#                             num_class = 12,
#                             max_depth = 11,
#                             alpha = 0,
#                             learning_rate = 0.13,
#                             n_estimators = 1200,
#                             seed = 42,
#                             )
# clf_XGb.fit(X_train, y_train)

In [ ]:
# joblib.dump(value = clf_XGb,
#             filename = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/clf_xgb.pkl')
clf_XGb = joblib.load(filename = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/clf_xgb.pkl')

In [ ]:
print("Accuracy on test: %.2f%%" % (clf_XGb.score(X = X_test, y = y_test) * 100.0))

In [ ]:
# ROC
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

for class_of_interest in range(12):
    class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
    metrics.RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        clf_XGb.predict_proba(X_test)[:, class_id],
        name = f"{class_of_interest} vs the rest",
        color = "darkorange",
        plot_chance_level = True,
        )
    plt.axis("square")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
    plt.legend()
    plt.show()

In [ ]:
cls_prob = clf_XGb.predict_proba(X = X_test, # (numpy array, pandas DataFrame, H2O DataTable's Frame , scipy.sparse, list of lists of int or float of shape = [n_samples, n_features]) – Input features matrix.
                                 )
cls_prob = pd.DataFrame(data = cls_prob, columns = labels)
cls_prob.to_csv(path_or_buf = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/cls_prob_xgb.csv',
                sep = ',', index = False)

In [ ]:
pred_y_test = clf_XGb.predict(X = X_test)
cm_test = confusion_matrix(y_true = y_test,
                           y_pred = pred_y_test,
                           labels = list(labels.values()),
                           normalize = 'true')
cm_test = pd.DataFrame(data = cm_test, index = list(labels.keys()), columns = list(labels.keys()))

colorcode_confusion_matrix(df_cm = cm_test, title = 'Confusion matrix on test set')

### Feature Importance

In [ ]:
# =============================================================================
# Shapley Values for Feature Importance
# =============================================================================
shap.initjs()
explainer = shap.TreeExplainer(clf_XGb, approximate = True)
shap_values = explainer.shap_values(X_test)#.sample(frac = 0.1))

In [ ]:
def get_swap_dict(d):
    return {v: k for k, v in d.items()}
swap_labels = get_swap_dict(labels)
newCmap = ['#a6cee3','#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c',
           '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928']

newCmap = LinearSegmentedColormap.from_list(name = '',
                                            colors = newCmap,
                                            N = 12)


In [ ]:
feat_dict = {
             'Popo_1km': 'Pop_1km',
             '11': 'Open Water',
             '12': 'Snow/Ice',
             '13': 'Developed-Upland Deciduous Forest',
             '14': 'Developed-Upland Evergreen Forest',
             '15': 'Developed-Upland Mixed Forest',
             '16': 'Developed-Upland Herbaceous',
             '17': 'Developed-Upland Shrubland',
             '22': 'Developed - Low Intensity',
             '23': 'Developed - Medium Intensity',
             '24': 'Developed - High Intensity',
             '25': 'Developed-Roads',
             '31': 'Barren',
             '32': 'Quarries-Strip Mines-Gravel Pits-Well and Wind Pads',
             '61': 'NASS-Vineyard',
             '63': 'NASS-Row Crop-Close Grown Crop',
             '64': 'NASS-Row Crop',
             '65': 'NASS-Close Grown Crop',
             '68': 'NASS-Wheat',
             '69': 'NASS-Aquaculture',
             '100': 'Sparse Vegetation Canopy',
             '200': 'Shrub Cover',
             '300': 'Herb Cover',
            }
feat_name = X_train.columns.to_list()
for idx, item in enumerate(X_train.columns):
    if item in feat_dict:
        feat_name[idx] = feat_dict[item]

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
plt_shap = shap.summary_plot(shap_values,                       # Use Shap values array
                             features = X_train,                # Use training set features
                             feature_names = feat_name,         # Use column names
                             show = False,                      # Set to false to output to folder
                             class_names = swap_labels,         # Converting the class from int to orginal labels [Dict class]
                             color = newCmap,                   # New set of color, it should be in 'LinearSegmentedColormap' class
                             color_bar_label = 'Feature value',
                             plot_size = (15,5),                # Change plot size
                            #  class_inds = 'original',         # It will always keep the class labels in the same order
                             )
plt.savefig(f'/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/Plots/summary_plot_West_xgb.png')
plt.show()

In [ ]:
feature_names = X_train.columns
sh_val = np.zeros(shape = shap_values[0].shape)

for cls in range(len(shap_values)):
    sh_val = sh_val + shap_values[cls]

rf_resultX = pd.DataFrame(sh_val, columns = feature_names)

vals = np.abs(rf_resultX.values).mean(0)

shap_importance = pd.DataFrame(list(zip(feature_names, vals)),
                                columns = ['col_name', 'feature_importance_vals'])
shap_importance.sort_values(by = ['feature_importance_vals'],
                            ascending = False,
                            inplace = True)
shap_importance.to_csv(path_or_buf = '/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/feat_import_xgb.csv',
                       sep = ',',
                       index = False)

In [ ]:
for i in range(12):
    shap.summary_plot(shap_values = shap_values[i],     # Use Shap values array
                      plot_type = 'violin',             # “dot” (default for single output), “bar” (default for multi-output), “violin”
                      features = X_test,               # Use training set features
                      feature_names = feat_name,        # Use column names
                      show = False,                     # Set to false to output to folder
                      plot_size = (20, 10),                 # Change plot size: None, 'auto', (10, 7)
                      cmap = "plasma",
                      # title = f'feature importance for {swap_labels[0]} cause',
                      )
    ax = plt.gca()
    ax.set_xlim(-1, 1) 
    plt.title(label = f'Feature importance for {swap_labels[i]} cause',
              loc = 'center',
            #   pad = 15,
              )
    nam = swap_labels[i].replace('/', ' or ')
    plt.savefig(f'/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/Plots/West_xgb_{nam}.png')
    plt.clf()

In [ ]:
image_list = glob.glob(pathname = f'/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/Plots/West_xgb_*.png')

ii = [0, 2, 4, 6, 8, 10]
count = 0
Horizental_frame = [[],[],[],[],[],[]]
for i in ii:
    image1 = cv.imread(image_list[i])
    image2 = cv.imread(image_list[i + 1])
    Horizental_frame[count] = cv.hconcat((image1, image2)) # or vconcat for vertical concatenation
    count += 1
final_frame = cv.vconcat((Horizental_frame[0],
                          Horizental_frame[1],
                          Horizental_frame[2],
                          Horizental_frame[3],
                          Horizental_frame[4],
                          Horizental_frame[5]))

cv.imwrite(f'/bsuhome/yavarpourmohamad/scratch/Unknown_classification/Engineered Features/Plots/per_cause_West_xgb.png', final_frame)

In [ ]:
for i in image_list:
    os.remove(i)
print(f'All {len(image_list)} files deleted')

### Inference Plotting

In [ ]:
shap_feat = shap_importance.reset_index(drop = True).loc[0:19, 'col_name'].values
plt_dt = pd.concat([y_test, X_test], axis = 1)[np.append('NWCG_GENERAL_CAUSE', shap_feat)].sample(n = 50, random_state = 42)
plt_dt = plt_dt.set_index(keys = 'NWCG_GENERAL_CAUSE')
# Calculate the distance between each sample
Z = Sciplot.linkage(y = plt_dt,
                    method = 'ward')
# other methods: single, complete, average, weighted, centroid, median

# Plot with Custom leaves
Sciplot.dendrogram(Z = Z,
                   leaf_rotation = 90,
                   leaf_font_size = 8,
                   labels = plt_dt.index)

# Show the graph
plt.show()

In [ ]:
shap_feat = shap_importance.reset_index(drop = True).loc[0:19, 'col_name'].values
sctr_plt = pd.concat([y_test, X_test], axis = 1)[np.append('NWCG_GENERAL_CAUSE', shap_feat)].sample(n = 500, random_state = 42)
sctr_plt[sctr_plt['GHM'] < 0] = 0

for i in sctr_plt['NWCG_GENERAL_CAUSE'].unique():
        sctr_plt.loc[sctr_plt['NWCG_GENERAL_CAUSE'] == i, 'NWCG_GENERAL_CAUSE'] = swap_labels[i]

sns.set(rc = {'figure.figsize':(10, 5)})
ax  = sns.scatterplot(data = sctr_plt,
                  x = 'GHM',
                  y = 'Elevation_1km',
                  hue = 'NWCG_GENERAL_CAUSE',
                  alpha = 0.6)
sns.move_legend(ax, "upper left", bbox_to_anchor = (1, 1))
plt.xlabel(xlabel = 'Global Human Modification')
plt.ylabel(ylabel = 'Elevation (m)')
plt.show()

sns.set(rc = {'figure.figsize':(10, 5)})
ax  = sns.kdeplot(data = sctr_plt,
                  x = 'GHM',
                  y = 'Elevation_1km',
                  hue = 'NWCG_GENERAL_CAUSE',
                  shade = True,
                  alpha = 0.75)
sns.move_legend(ax, "upper left", bbox_to_anchor = (1, 1))
plt.xlabel(xlabel = 'Global Human Modification')
plt.ylabel(ylabel = 'Elevation (m)')
plt.show()

In [ ]:
sns.set(rc = {'figure.figsize':(10, 5)})
ax  = sns.scatterplot(data = sctr_plt,
                      x = 'DISCOVERY_DOY',
                      y = 'Elevation_1km',
                      hue = 'NWCG_GENERAL_CAUSE',
                    #   palette = newCmap,
                      alpha = 0.6)
sns.move_legend(ax, "upper left", bbox_to_anchor = (1, 1))
plt.xlabel(xlabel = 'Day of Year')
plt.ylabel(ylabel = 'Elevation (m)')
plt.show()

ax  = sns.kdeplot(data = sctr_plt,
                  x = 'DISCOVERY_DOY',
                  y = 'Elevation_1km',
                  hue = 'NWCG_GENERAL_CAUSE',
                  shade = True,
                  alpha = 0.75)
sns.move_legend(ax, "upper left", bbox_to_anchor = (1, 1))
plt.xlabel(xlabel = 'Day of Year')
plt.ylabel(ylabel = 'Elevation (m)')
plt.show()

In [ ]:
ax  = sns.scatterplot(data = sctr_plt,
                      x = 'DISCOVERY_DOY',
                      y = 'GHM',
                      hue = 'NWCG_GENERAL_CAUSE',
                    #   palette = newCmap,
                      alpha = 0.6)
sns.move_legend(ax, "upper left", bbox_to_anchor = (1, 1))
plt.xlabel(xlabel = 'Day of Year')
plt.ylabel(ylabel = 'Global Human Modification')
plt.show()

ax  = sns.kdeplot(data = sctr_plt,
                  x = 'DISCOVERY_DOY',
                  y = 'GHM',
                  hue = 'NWCG_GENERAL_CAUSE',
                  shade = True,
                  alpha = 0.75)
sns.move_legend(ax, "upper left", bbox_to_anchor = (1, 1))
plt.xlabel(xlabel = 'Day of Year')
plt.ylabel(ylabel = 'Global Human Modification')
plt.show()

In [ ]:
sns.pairplot(data = sctr_plt[['NWCG_GENERAL_CAUSE', 'DISCOVERY_DOY', 'Elevation_1km', 'GHM']], # .sample(n = 1000, random_state = 42),
             kind = 'scatter',
             hue = 'NWCG_GENERAL_CAUSE',
             plot_kws = dict(s = 10, edgecolor = 'white', linewidth = 0.8)
             )
plt.show()

In [ ]:
# Default plot
sns.clustermap(data = sctr_plt[['NWCG_GENERAL_CAUSE', 'DISCOVERY_DOY', 'Elevation_1km', 'GHM']].sample(n = 500, random_state = 42).set_index(keys = 'NWCG_GENERAL_CAUSE'),
            #    z_score = 1, # Either 0 (rows) or 1 (columns)
               standard_scale = 1, # Either 0 (rows) or 1 (columns)
               metric = 'euclidean',
               method = 'ward'
               )

# Show the graph
plt.show()

In [ ]:
for feat in ['GHM', 'Elevation_1km', 'DISCOVERY_DOY']:
    sns.violinplot(data = sctr_plt,
                x = 'NWCG_GENERAL_CAUSE',
                y = feat,
                order = list(labels.keys()),
                palette = sns.color_palette())
    plt.xticks(rotation = 70)
    plt.show()

In [ ]:
# for creating a responsive plot
# %matplotlib widget

# Import libraries
from mpl_toolkits import mplot3d

# Creating figure
fig = plt.figure(figsize = (16, 9))
# ax = plt.axes(projection = '3d')
ax = mplot3d.Axes3D(fig)

# Add x, y gridlines 
ax.grid(b = True,
        color ='black', 
		linestyle ='-.',
        linewidth = 0.3, 
		alpha = 0.2) 

sctr_plt_int = sctr_plt.copy()
for i in sctr_plt_int['NWCG_GENERAL_CAUSE'].unique():
        sctr_plt_int.loc[sctr_plt_int['NWCG_GENERAL_CAUSE'] == i, 'NWCG_GENERAL_CAUSE'] = labels[i]

sctr_plt_int = sctr_plt_int.sample(n = 500, random_state = 42)

# Creating plot
sctt = ax.scatter3D(xs = sctr_plt_int.GHM,
					ys = sctr_plt_int.Elevation_1km,
					zs = sctr_plt_int.DISCOVERY_DOY,
					alpha = 0.8,
					c = sctr_plt_int.NWCG_GENERAL_CAUSE, 
					cmap = newCmap,
					marker = 'o')

plt.title('GHM vs Elevation vs Discovery doy')

ax.set_xlabel('Global Human Modification', fontweight ='bold') 
ax.set_ylabel('Elevation', fontweight ='bold') 
ax.set_zlabel('Discovery day of year', fontweight ='bold')
fig.colorbar(sctt, ax = ax, shrink = 0.5, aspect = 5)

# show plot
plt.show()